In [5]:
import pandas as pd
import cobra.flux_analysis
import time
from functools import partial
from src.mp_functions import combinations_subset, parallelize_dataframe, knockout_FBA


"""A mess of a document with different code cells.
Good to to use for any testing that involves the Recon3D model as it takes some time to load in."""

start_time = time.time()
model_file_path = 'C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/Human-GEM.mat'
model = cobra.io.load_matlab_model(model_file_path)

end_time = time.time()
print('Model load time: %.6f seconds' % (end_time - start_time))

Model load time: 43.273007 seconds


In [2]:
SNP_results = pd.read_table('C:/Users/Sigve/Genome_Data/results/SNPs_non_synonymous.tsv', index_col=0)

# Prepare SNP combinations; lists genes to knock out by model id.
combinations = pd.read_table('C:/Users/Sigve/Genome_Data/results/SNP_combinations.tsv', index_col=0)
combinations['gene_ids'] = combinations['gene_ids'].apply(lambda x: x.split(';'))


C:\Users\Sigve\miniconda3\envs\Master\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
# FBA code cell
start_time = time.time()


# Does not work if model imported as sbml for some reason
combinations = parallelize_dataframe(combinations, partial(combinations_subset, partial(knockout_FBA, model)), 8)

# Code for single thread FBA
#combinations['results'] = combinations['gene_ids'].apply(partial(knockout_FBA, model))


combinations.to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/results/test.tsv', sep='\t')

end_time = time.time()
print('FBA run time: %.6f seconds' % (end_time - start_time))

FBA run time: 172.728217 seconds


In [5]:
# Check for non nominal results
for i, result in combinations.iterrows():
    res = result['results'].objective_value
    if int(res) < 187:
        print(result)

SNP_ids     rs9827;rs4988323;rs547552367;rs182737113;rs190...
gene_ids    [ENSG00000078237, ENSG00000124789, ENSG0000011...
results                     <Solution 0.000 at 0x1ad370f3940>
Name: 1, dtype: object
SNP_ids     rs531900792;rs187915150;rs61752937;rs61753543;...
gene_ids    [ENSG00000133706, ENSG00000162434, ENSG0000012...
results                     <Solution 0.000 at 0x1ad3ad204f0>
Name: 4, dtype: object
SNP_ids     rs546148482;rs117145430;rs9997664;rs140078589;...
gene_ids    [ENSG00000110328, ENSG00000139370, ENSG0000012...
results                     <Solution 0.000 at 0x1ad3ad07e80>
Name: 7, dtype: object
SNP_ids     rs181476100;rs189865294;rs548917691;rs74583923...
gene_ids    [ENSG00000130816, ENSG00000031698, ENSG0000017...
results                     <Solution 0.000 at 0x1ad3ad09b20>
Name: 10, dtype: object
SNP_ids     rs138534343;rs149122082;rs140611183;rs56466485...
gene_ids    [ENSG00000010219, ENSG00000117543, ENSG0000015...
results                     <Solution 0

In [6]:
gene_lists = combinations['gene_ids'].to_list()
gene_lists = [ids for sublist in gene_lists for ids in sublist]

counts = pd.Series(gene_lists).value_counts()
counts

ENSG00000155657    18
ENSG00000159063    13
ENSG00000142875    13
ENSG00000185803    11
ENSG00000126883     9
                   ..
ENSG00000060762     1
ENSG00000198919     1
ENSG00000136872     1
ENSG00000075975     1
ENSG00000147548     1
Length: 537, dtype: int64

In [3]:
# Test stuff

#print(model.reactions.MAR04908.genes)
#print(model.reactions.MAR04908.gene_reaction_rule)
#print(model.genes.get_by_id('ENSG00000103024'))

#model.genes.ENSG00000103024.knock_out()
gene_id = ['ENSG00000155657','ENSG00000159063','ENSG00000142875']

genes = [model.genes.get_by_id(i) for i in gene_id]
#gene = model.genes.get_by_id('ENSG00000227232')
genes


AttributeError: DictList has no attribute or entry ENSG00000227232

In [3]:
# Read in processed SNP data
path = 'C:/Users/Sigve/Genome_Data/'
SNP_file = path + 'results/SNPs_for_FBA.tsv'
SNP_df = pd.read_table(SNP_file)

# Get gene list
genes = [model.genes.get_by_id(i) for i in set(SNP_df['gene_id'].to_list())]
print(len(genes))

single_deletion_results = cobra.flux_analysis.single_gene_deletion(model, genes, method='fba', processes=8)
#double_deletion_results = cobra.flux_analysis.double_gene_deletion(model, genes, method='fba', processes=8)

single_deletion_results.to_csv(path_or_buf=path + 'results/test_single.tsv', sep='\t')
#double_deletion_results.to_csv(path_or_buf=path + 'results/test_double.tsv', sep='\t')

1311


In [13]:
# Remove single genes deletions that break the model before doing double deletion.
print(single_deletion_results.shape[0])
single_deletion_results = single_deletion_results[single_deletion_results['growth'] != 0]
genes = single_deletion_results['ids'].to_list()
print(genes)
genes2 = [item for sublist in genes for item in sublist]
print(len(genes2))
print(genes2)

1261
[{'ENSG00000052802'}, {'ENSG00000124006'}, {'ENSG00000197249'}, {'ENSG00000092068'}, {'ENSG00000146648'}, {'ENSG00000114738'}, {'ENSG00000165282'}, {'ENSG00000174886'}, {'ENSG00000169375'}, {'ENSG00000156515'}, {'ENSG00000079215'}, {'ENSG00000160752'}, {'ENSG00000164116'}, {'ENSG00000111817'}, {'ENSG00000152952'}, {'ENSG00000173540'}, {'ENSG00000184470'}, {'ENSG00000166484'}, {'ENSG00000149925'}, {'ENSG00000010404'}, {'ENSG00000152642'}, {'ENSG00000168487'}, {'ENSG00000112541'}, {'ENSG00000100596'}, {'ENSG00000177239'}, {'ENSG00000165349'}, {'ENSG00000196090'}, {'ENSG00000138061'}, {'ENSG00000017260'}, {'ENSG00000143199'}, {'ENSG00000108474'}, {'ENSG00000137700'}, {'ENSG00000145284'}, {'ENSG00000126522'}, {'ENSG00000060069'}, {'ENSG00000043514'}, {'ENSG00000171100'}, {'ENSG00000198886'}, {'ENSG00000139428'}, {'ENSG00000137868'}, {'ENSG00000198727'}, {'ENSG00000116096'}, {'ENSG00000171503'}, {'ENSG00000133216'}, {'ENSG00000069431'}, {'ENSG00000068438'}, {'ENSG00000160868'}, {'ENSG0

In [11]:
SNPs = pd.read_table('C:/Users/Sigve/Genome_Data/SNP_data/ALS_variants/ALS_15_loci.tsv')
SNPs.drop(columns='var_src', inplace=True)
SNPs['ancestral'] = SNPs['var'].apply(lambda x: x[0])
SNPs['var'] = SNPs['var'].apply(lambda x: x.split('/')[1:])
SNPs = SNPs.explode('var', ignore_index=True)
SNPs['var'] = SNPs['ancestral'] + '/' + SNPs['var']
SNPs.drop(columns='ancestral', inplace=True)
print(SNPs.head())

SNPs.to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/SNP_data/ALS_variants/ALS_15_loci_edit.tsv', sep='\t')

       snp_id  chr        pos  var
0  rs10280711    7  157689088  G/A
1  rs10280711    7  157689088  G/C
2  rs10280711    7  157689088  G/T
3  rs10463311    5  151031274  C/A
4  rs10463311    5  151031274  C/G


In [25]:
with model:
    #model.genes.get_by_id('ENSG00000160226').knock_out()
    #model.genes.get_by_id('ENSG00000142168').knock_out()
    print(model.optimize(objective_sense='maximize'))

KeyError: 'ENSG00000142168'

In [21]:
phewas = pd.read_csv('C:/Users/Sigve/Genome_Data/SNP_data/phewas/phewas-catalog.csv')

print(phewas.head())

phewas_ids = phewas['snp']
phewas_ids = phewas_ids.unique()
phewas_ids = pd.Series(phewas_ids)

print(type(phewas_ids))
print(phewas_ids[0:10])
print(len(phewas_ids))
phewas_ids.to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/SNP_data/phewas/phewas_ids.tsv', sep='\t', index=False)

    chromosome         snp                          phewas phenotype  cases  \
0  19 45395619   rs2075650                       Alzheimer's disease    737   
1  19 45395619   rs2075650                                 Dementias   1170   
2     6 396321  rs12203592                         Actinic keratosis   2505   
3   6 26093141   rs1800562                  Iron metabolism disorder     40   
4  19 45395619   rs2075650  Delirium dementia and amnestic disorders   1566   

        p-value  odds-ratio gene_name  phewas code  \
0  5.237000e-28       2.410    TOMM40       290.11   
1  2.409000e-26       2.114    TOMM40       290.10   
2  4.141000e-26       1.691      IRF4       702.10   
3  3.409000e-25      12.270       HFE       275.10   
4  8.027000e-24       1.841    TOMM40       290.00   

                                   gwas-associations  
0  Alzheimer's disease, Alzheimer's disease bioma...  
1  Alzheimer's disease, Alzheimer's disease bioma...  
2  Eye color, Hair color, Freckling

In [75]:
phewas = pd.read_csv('C:/Users/Sigve/Genome_Data/SNP_data/phewas/phewas-catalog.csv')
phewas_var = pd.read_table('C:/Users/Sigve/Genome_Data/SNP_data/phewas/phewas_variants.tsv')

phewas = phewas.merge(phewas_var, left_on='snp', right_on='Variant name')
#phewas = phewas[['snp', 'Chromosome/scaffold name','Chromosome/scaffold position start (bp)', 'Variant alleles', 'phewas code']]
phewas.rename(columns={'snp': 'snp_id','Chromosome/scaffold name': 'chr', 'Variant alleles': 'var', 'Chromosome/scaffold position start (bp)': 'pos', 'phewas code': 'phewas_code'}, inplace=True)

phewas['phewas_code'] = phewas['phewas_code'].apply(lambda x: str(x))
#phewas = phewas.groupby(['snp', 'chr', 'pos', 'var'])['phewas_code'].apply(', '.join).reset_index()

reference_phewas = phewas.drop(columns=['Variant name', 'chromosome', 'cases', 'gwas-associations'])
phewas = reference_phewas[['snp_id', 'chr', 'pos', 'var']].drop_duplicates(subset=['snp_id']).sort_values(by=['chr', 'pos'])

phewas['ancestral'] = phewas['var'].apply(lambda x: x[0])
phewas['var'] = phewas['var'].apply(lambda x: x.split('/')[1:])
phewas = phewas.explode('var', ignore_index=True)
phewas['var'] = phewas['ancestral'] + '/' + phewas['var']
phewas.drop(columns='ancestral', inplace=True)

phewas.reset_index(inplace=True, drop=True)

print(phewas.shape[0])

#reference_phewas.to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/SNP_data/phewas/phewas_ref.tsv', sep='\t', index=False)
#phewas.to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/SNP_data/phewas/phewas_extracted_all.tsv', sep='\t', index=False)
#temp = reference_phewas[reference_phewas['phewas_code']==290.11]
#temp.sort_values(by=['chr', 'pos'], inplace=True)
#temp.to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/SNP_data/phewas/test.tsv', sep='\t', index=False)

res = phewas[phewas['snp_id'].isin(reference_phewas[reference_phewas['phewas_code'].str.contains('290.\d*')]['snp_id'])]

res.to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/SNP_data/phewas/phewas_extracted_290.tsv', sep='\t', index=False)
print(res.head())


5002
      snp_id chr      pos  var
4  rs3748816   1  2595307  A/G
5  rs3748816   1  2595307  A/T
7  rs2651899   1  3167148  T/A
8  rs2651899   1  3167148  T/C
9   rs966321   1  4255144  G/C
